<a href="https://colab.research.google.com/github/Ataerman/LLM_RAG-AI_AGENT/blob/main/First_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-openai langchain-cohere langchain-community \
sentence-transformers faiss-cpu duckduckgo-search \
datasets accelerate \
tiktoken \
cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/14

In [2]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
openai_api_key=userdata.get('openai')
openai_llm=ChatOpenAI(model='gpt-3.5-turbo', #creating llm model (using chatgpt 3.5 with api)
                      openai_api_key=openai_api_key,
                      temperature=0.3)

In [3]:
from langchain import PromptTemplate
react_template="""Answer the following questions as best you can.You have access to the following tools:
{tools}
Use the following format :
Question:the input question you must answer
Thought:you should always think about what to do
Action:the action to take,should be one of [{tool_names}]
Action Input:the input to the action
Observation:the result of the action
...(this Thought/Action/Action Input/Observation can repeat N times)
Thought:I know the final answer
Final Answer:the final answer to the original input question

Begin!

Question:{input}
Thought:{agent_scratchpad}"""
prompt=PromptTemplate(template=react_template,
                      input_variables=['tools','tool_names','input','agent_scratchpad'])


In [5]:
from langchain.agents import load_tools,Tool
from langchain.tools import DuckDuckGoSearchResults
search=DuckDuckGoSearchResults()
search_tool=Tool(
    name='duckduck',
    description='A web search engine.Use this to as a search engine for general queries',
    func=search.run
)
tools=load_tools(['llm-math'],llm=openai_llm)
tools.append(search_tool)

In [7]:
from langchain.agents import AgentExecutor,create_react_agent
agent=create_react_agent(openai_llm,tools,prompt)
agent_executer=AgentExecutor(
    agent=agent,tools=tools,verbose=True,handle_parsing_errors=True
)

In [8]:
agent_executer({"input":"What is the current price of an iphone16 Pro in USD? How much would it cost in TRY if the exchange rate is 38 TRY for 1 USD?"})

<ipython-input-8-1e23044205d5>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executer({"input":"What is the current price of an iphone16 Pro in USD? How much would it cost in TRY if the exchange rate is 38 TRY for 1 USD?"})




> Entering new AgentExecutor chain...
I should use a web search engine to find the current price of an iPhone 16 Pro in USD and then use a calculator to convert it to TRY.
Action:duckduck
Action Input:current price of iPhone 16 Pro in USDsnippet: The ‌ iPhone 16 Pro ‌ starts at $999 for 128GB and $1,199 for ‌‌iPhone 16 Pro‌‌ Max for 256GB. Here's a breakdown of Apple's prices, including UK equivalents. iPhone 16e 128GB: $599 ..., title: How Much Does Each iPhone 16 Model Cost? - MacRumors, link: https://www.macrumors.com/2025/02/21/how-much-does-iphone-16-cost/, snippet: The iPhone 16 Pro and iPhone 16 Pro Max also have a 1 TB tier, which is $200/£200/€250 more than the 512 GB variant. There is no 128 GB version of the Max., title: Here are the Apple iPhone 16 and iPhone 16 Pro prices around the world, link: https://www.gsmarena.com/here_are_the_apple_iphone_16_and_iphone_16_pro_prices_around_the_world-news-64467.php, snippet: Preorders for the iPhone 16 start September 13, and here 

{'input': 'What is the current price of an iphone16 Pro in USD? How much would it cost in TRY if the exchange rate is 38 TRY for 1 USD?',
 'output': 'The current price of an iPhone 16 Pro in USD is $999. It would cost 37,962 TRY with an exchange rate of 38 TRY for 1 USD.'}